# Consolidate data (Batch1)

<div style="text-align: right"> Author: Vassil Dimitrov </div>
<div style="text-align: right"> Date: 2023-08-01 </div>

Batch 1 of the data will be prepared by merging the patients_batch1 data with the corresponding patient data from all other available tables. Note that some preprocessing will be required for some of the tables encluding `medications`, `observations` and `procedures`.

## Prep

### Load libraries

In [41]:
import numpy as np
import pandas as pd

---

### Load patients data

In [ ]:
b = pd.read_pickle('patients_batch1.pkl')

In [ ]:
b.shape

In [ ]:
display(b.head())

Obtain column names from the patients dataframe for downstream analyses.

In [ ]:
patients_dat_cols = list(b.columns)
print(patients_dat_cols)

---

## Functions

### `PATIENT` to index & explode

In [121]:
def patient_subset_index_explode(df1, df2):
    # Subset data
    print('subsetting...')
    tab = df2[df2['PATIENT'].isin(df1.index)]
    del df2
    print('Subset table dimensions:', tab.shape)
   
    # Create a new column 'encounter' to differentiate between duplicate rows for each patient
    print('Counting number of encounters per patient...')
    tab['encounter_n'] = tab.groupby('PATIENT').cumcount()+1
    print('Pivoting table and setting "PATIENT" as index')
    tab = tab.pivot_table(index='PATIENT',
                          columns='encounter_n',
                          aggfunc='first')
    # Flatten the MultiIndex columns
    tab.columns = [f'{col[0]}_{col[1]}' for col in tab.columns]
    # Fill in nulls with 0
    print('Filling in nulls with 0s')
    tab.fillna(0, inplace=True)
    # Convert to uint32
    tab = tab.astype('uint32')
    # Return processed and subset table
    return tab

### Batch merge

In [116]:
def batch_merge (df1, tab):
    # Merge the data with consolidated dataframe
    print('Merging...')
    df1 = df1.merge(tab, 
                  left_index=True, 
                  right_index=True, 
                  how='left')
    print('Merged!')
    print('New dimensions:', df1.shape)
    return df1

### Load data

In [44]:
def load_table (tab_name):
    to_merge = pd.read_pickle(tab_name)
    print(f'Dimensions for {tab_name}:')
    print(to_merge.shape)
    return to_merge

### Subset batch

In [45]:
def subset_batch_merge (df1, df2):
    print('Merging...')
    df_b = df2[df2.index.isin(df1.index)]
    df1 = df1.merge(df_b, 
                  left_index=True, 
                  right_index=True, 
                  how='left')
    print('Merged!')
    return df1

### Tidy up

In [ ]:
def tidy_up (df, cols2exclude):
    print('Filling up with 0s')
    df = df.fillna(0)
    cols = [col for col in list(df.columns) if col not in cols2exclude]
    for col in cols:
        print(f'Processing column {col}...')
        if pd.api.types.is_sparse(df[col]):
            df[col] = df[col].sparse.to_dense().astype(np.uint32)
        else:
            df[col] = df[col].astype(np.uint32)
        print(f'Processed column {col}.')
    return df

### Process and merge
Uses the three other functions defined above: `load_table`, `subset_batch_merge` and `tidy_up`.

In [107]:
def process_n_merge_processed (df_a, tab_name_b):
    cols2exclude=list(df_a.columns)
    df_b = load_table(tab_name_b)
    df_a = subset_batch_merge(df_a, df_b)
    df_a = tidy_up(df_a, cols2exclude)
    return df_a

---
---

---
---

## Allergies

`allergies` is already preprocessed so it will essentially be loaded, the corresponding patients will be identified based on the `patients_batch1` information, and their respecive data will be merged to the patients data based on the indexes of both tables, keeping all batch1 patients.

### Read table

In [ ]:
to_merge = pd.read_pickle('allergies_encounters.pkl')
# Sanity check
print('dimensions:', to_merge.shape)
display(to_merge.head())

### Subset for batch1 patients only

In [ ]:
to_merge_b = to_merge[to_merge.index.isin(b.index)]
# Sanity check
print('dimensions:', to_merge_b.shape)
display(to_merge_b.head())

A bit over 12,000 patients from batch1 also have information about allergies.

### Merge `allergies` to batch1

In [ ]:
batch = b.merge(to_merge_b, 
                  left_index=True, 
                  right_index=True, 
                  how='left')
# Sanity check:
print(batch.shape)
display(batch.head())

In [ ]:
# Clean up memory
del b, to_merge, to_merge_b

### Tidy up table

Replace nulls with 0 (no allergy encounter with a physician).

In [ ]:
batch.fillna(0, inplace=True)
display(batch.head())

Check the variable types for each column.

In [ ]:
batch.dtypes

The allergy types were transformed into `float64` type. They will be converted to `uint32` to save memory.

In [ ]:
for col in batch.columns:
    if col not in patients_dat_cols:
        print('Converting', col)
        batch[col] = batch[col].astype('uint32')
    else:
        continue

In [ ]:
batch.dtypes

The types for the merged dataframe are of appropriate value.  
  
  The column names will be saved to exclude for downstream data manipulation.

In [ ]:
cols2exclude = list(batch.columns)
print(cols2exclude)

---
---

## Careplans

In [48]:
careplans = load_table('careplans_encounters.pkl')

Dimensions: (1188051, 2888)
(1188051, 2888)


In [53]:
careplans.dtypes

Administration of intravenous fluids_1    Sparse[uint32, 0]
Administration of intravenous fluids_2    Sparse[uint32, 0]
Administration of intravenous fluids_3    Sparse[uint32, 0]
Administration of intravenous fluids_4    Sparse[uint32, 0]
Administration of intravenous fluids_5    Sparse[uint32, 0]
                                                ...        
Wound care_34                             Sparse[uint32, 0]
Wound care_35                             Sparse[uint32, 0]
Wound care_36                             Sparse[uint32, 0]
Wound care_37                             Sparse[uint32, 0]
Wound care_38                             Sparse[uint32, 0]
Length: 2888, dtype: object

In [50]:
print(batch.shape)
cols2exclude = list(batch.columns)
batch = subset_batch_merge(batch, careplans)
print(batch.shape)

(119293, 224)
(119293, 3112)


In [82]:
batch.dtypes

MARITAL                       uint8
GENDER                        uint8
m_since_birth                uint16
RACE_asian                    uint8
RACE_black                    uint8
                        ...        
Wound care_34    Sparse[float64, 0]
Wound care_35    Sparse[float64, 0]
Wound care_36    Sparse[float64, 0]
Wound care_37    Sparse[float64, 0]
Wound care_38    Sparse[float64, 0]
Length: 3112, dtype: object

In [103]:
batch = tidy_up(batch, cols2exclude)
batch.dtypes

Administration of intravenous fluids_1
Administration of intravenous fluids_2
Administration of intravenous fluids_3
Administration of intravenous fluids_4
Administration of intravenous fluids_5
Administration of intravenous fluids_6
Administration of intravenous fluids_7
Administration of intravenous fluids_8
Administration of intravenous fluids_9
Administration of intravenous fluids_10
Administration of intravenous fluids_11
Administration of intravenous fluids_12
Administration of intravenous fluids_13
Administration of intravenous fluids_14
Administration of intravenous fluids_15
Administration of intravenous fluids_16
Administration of intravenous fluids_17
Administration of intravenous fluids_18
Administration of intravenous fluids_19
Administration of intravenous fluids_20
Administration of intravenous fluids_21
Administration of intravenous fluids_22
Administration of intravenous fluids_23
Administration of intravenous fluids_24
Administration of intravenous fluids_25
Administr

Anti-suicide psychotherapy_24
Anti-suicide psychotherapy_25
Anti-suicide psychotherapy_26
Anti-suicide psychotherapy_27
Anti-suicide psychotherapy_28
Anti-suicide psychotherapy_29
Anti-suicide psychotherapy_30
Anti-suicide psychotherapy_31
Anti-suicide psychotherapy_32
Anti-suicide psychotherapy_33
Anti-suicide psychotherapy_34
Anti-suicide psychotherapy_35
Anti-suicide psychotherapy_36
Anti-suicide psychotherapy_37
Anti-suicide psychotherapy_38
Application of back brace_1
Application of back brace_2
Application of back brace_3
Application of back brace_4
Application of back brace_5
Application of back brace_6
Application of back brace_7
Application of back brace_8
Application of back brace_9
Application of back brace_10
Application of back brace_11
Application of back brace_12
Application of back brace_13
Application of back brace_14
Application of back brace_15
Application of back brace_16
Application of back brace_17
Application of back brace_18
Application of back brace_19
Applicat

Cancer care plan_22
Cancer care plan_23
Cancer care plan_24
Cancer care plan_25
Cancer care plan_26
Cancer care plan_27
Cancer care plan_28
Cancer care plan_29
Cancer care plan_30
Cancer care plan_31
Cancer care plan_32
Cancer care plan_33
Cancer care plan_34
Cancer care plan_35
Cancer care plan_36
Cancer care plan_37
Cancer care plan_38
Cancer education_1
Cancer education_2
Cancer education_3
Cancer education_4
Cancer education_5
Cancer education_6
Cancer education_7
Cancer education_8
Cancer education_9
Cancer education_10
Cancer education_11
Cancer education_12
Cancer education_13
Cancer education_14
Cancer education_15
Cancer education_16
Cancer education_17
Cancer education_18
Cancer education_19
Cancer education_20
Cancer education_21
Cancer education_22
Cancer education_23
Cancer education_24
Cancer education_25
Cancer education_26
Cancer education_27
Cancer education_28
Cancer education_29
Cancer education_30
Cancer education_31
Cancer education_32
Cancer education_33
Cancer ed

Delivery of rehabilitation for spinal cord injury_6
Delivery of rehabilitation for spinal cord injury_7
Delivery of rehabilitation for spinal cord injury_8
Delivery of rehabilitation for spinal cord injury_9
Delivery of rehabilitation for spinal cord injury_10
Delivery of rehabilitation for spinal cord injury_11
Delivery of rehabilitation for spinal cord injury_12
Delivery of rehabilitation for spinal cord injury_13
Delivery of rehabilitation for spinal cord injury_14
Delivery of rehabilitation for spinal cord injury_15
Delivery of rehabilitation for spinal cord injury_16
Delivery of rehabilitation for spinal cord injury_17
Delivery of rehabilitation for spinal cord injury_18
Delivery of rehabilitation for spinal cord injury_19
Delivery of rehabilitation for spinal cord injury_20
Delivery of rehabilitation for spinal cord injury_21
Delivery of rehabilitation for spinal cord injury_22
Delivery of rehabilitation for spinal cord injury_23
Delivery of rehabilitation for spinal cord injury_

Drug addiction counseling_32
Drug addiction counseling_33
Drug addiction counseling_34
Drug addiction counseling_35
Drug addiction counseling_36
Drug addiction counseling_37
Drug addiction counseling_38
Drug addiction therapy_1
Drug addiction therapy_2
Drug addiction therapy_3
Drug addiction therapy_4
Drug addiction therapy_5
Drug addiction therapy_6
Drug addiction therapy_7
Drug addiction therapy_8
Drug addiction therapy_9
Drug addiction therapy_10
Drug addiction therapy_11
Drug addiction therapy_12
Drug addiction therapy_13
Drug addiction therapy_14
Drug addiction therapy_15
Drug addiction therapy_16
Drug addiction therapy_17
Drug addiction therapy_18
Drug addiction therapy_19
Drug addiction therapy_20
Drug addiction therapy_21
Drug addiction therapy_22
Drug addiction therapy_23
Drug addiction therapy_24
Drug addiction therapy_25
Drug addiction therapy_26
Drug addiction therapy_27
Drug addiction therapy_28
Drug addiction therapy_29
Drug addiction therapy_30
Drug addiction therapy_31


Ice therapy_16
Ice therapy_17
Ice therapy_18
Ice therapy_19
Ice therapy_20
Ice therapy_21
Ice therapy_22
Ice therapy_23
Ice therapy_24
Ice therapy_25
Ice therapy_26
Ice therapy_27
Ice therapy_28
Ice therapy_29
Ice therapy_30
Ice therapy_31
Ice therapy_32
Ice therapy_33
Ice therapy_34
Ice therapy_35
Ice therapy_36
Ice therapy_37
Ice therapy_38
Inhaled steroid therapy_1
Inhaled steroid therapy_2
Inhaled steroid therapy_3
Inhaled steroid therapy_4
Inhaled steroid therapy_5
Inhaled steroid therapy_6
Inhaled steroid therapy_7
Inhaled steroid therapy_8
Inhaled steroid therapy_9
Inhaled steroid therapy_10
Inhaled steroid therapy_11
Inhaled steroid therapy_12
Inhaled steroid therapy_13
Inhaled steroid therapy_14
Inhaled steroid therapy_15
Inhaled steroid therapy_16
Inhaled steroid therapy_17
Inhaled steroid therapy_18
Inhaled steroid therapy_19
Inhaled steroid therapy_20
Inhaled steroid therapy_21
Inhaled steroid therapy_22
Inhaled steroid therapy_23
Inhaled steroid therapy_24
Inhaled steroid 

Physical exercises_22
Physical exercises_23
Physical exercises_24
Physical exercises_25
Physical exercises_26
Physical exercises_27
Physical exercises_28
Physical exercises_29
Physical exercises_30
Physical exercises_31
Physical exercises_32
Physical exercises_33
Physical exercises_34
Physical exercises_35
Physical exercises_36
Physical exercises_37
Physical exercises_38
Physical therapy procedure_1
Physical therapy procedure_2
Physical therapy procedure_3
Physical therapy procedure_4
Physical therapy procedure_5
Physical therapy procedure_6
Physical therapy procedure_7
Physical therapy procedure_8
Physical therapy procedure_9
Physical therapy procedure_10
Physical therapy procedure_11
Physical therapy procedure_12
Physical therapy procedure_13
Physical therapy procedure_14
Physical therapy procedure_15
Physical therapy procedure_16
Physical therapy procedure_17
Physical therapy procedure_18
Physical therapy procedure_19
Physical therapy procedure_20
Physical therapy procedure_21
Physi

Recommendation to rest_33
Recommendation to rest_34
Recommendation to rest_35
Recommendation to rest_36
Recommendation to rest_37
Recommendation to rest_38
Respiratory therapy_1
Respiratory therapy_2
Respiratory therapy_3
Respiratory therapy_4
Respiratory therapy_5
Respiratory therapy_6
Respiratory therapy_7
Respiratory therapy_8
Respiratory therapy_9
Respiratory therapy_10
Respiratory therapy_11
Respiratory therapy_12
Respiratory therapy_13
Respiratory therapy_14
Respiratory therapy_15
Respiratory therapy_16
Respiratory therapy_17
Respiratory therapy_18
Respiratory therapy_19
Respiratory therapy_20
Respiratory therapy_21
Respiratory therapy_22
Respiratory therapy_23
Respiratory therapy_24
Respiratory therapy_25
Respiratory therapy_26
Respiratory therapy_27
Respiratory therapy_28
Respiratory therapy_29
Respiratory therapy_30
Respiratory therapy_31
Respiratory therapy_32
Respiratory therapy_33
Respiratory therapy_34
Respiratory therapy_35
Respiratory therapy_36
Respiratory therapy_37
Re

Stretching exercises_9
Stretching exercises_10
Stretching exercises_11
Stretching exercises_12
Stretching exercises_13
Stretching exercises_14
Stretching exercises_15
Stretching exercises_16
Stretching exercises_17
Stretching exercises_18
Stretching exercises_19
Stretching exercises_20
Stretching exercises_21
Stretching exercises_22
Stretching exercises_23
Stretching exercises_24
Stretching exercises_25
Stretching exercises_26
Stretching exercises_27
Stretching exercises_28
Stretching exercises_29
Stretching exercises_30
Stretching exercises_31
Stretching exercises_32
Stretching exercises_33
Stretching exercises_34
Stretching exercises_35
Stretching exercises_36
Stretching exercises_37
Stretching exercises_38
Suicide precautions_1
Suicide precautions_2
Suicide precautions_3
Suicide precautions_4
Suicide precautions_5
Suicide precautions_6
Suicide precautions_7
Suicide precautions_8
Suicide precautions_9
Suicide precautions_10
Suicide precautions_11
Suicide precautions_12
Suicide precau

MARITAL           uint8
GENDER            uint8
m_since_birth    uint16
RACE_asian        uint8
RACE_black        uint8
                  ...  
Wound care_34    uint32
Wound care_35    uint32
Wound care_36    uint32
Wound care_37    uint32
Wound care_38    uint32
Length: 3112, dtype: object

In [104]:
print('Dimensions so far:', batch.shape)

Dimensions so far: (119293, 3112)


In [105]:
# Clean up memory
del careplans, cols2exclude

---
---

## Conditions

In [108]:
batch = process_n_merge_processed (batch, 'conditions_encounters.pkl')
print('Dimensions so far:', batch.shape)

Dimensions: (1373324, 2816)
Acute allergic reaction_1
Acute allergic reaction_2
Acute allergic reaction_3
Acute allergic reaction_4
Acute allergic reaction_5
Acute allergic reaction_6
Acute allergic reaction_7
Acute allergic reaction_8
Acute allergic reaction_9
Acute allergic reaction_10
Acute allergic reaction_11
Acute allergic reaction_12
Acute allergic reaction_13
Acute allergic reaction_14
Acute allergic reaction_15
Acute allergic reaction_16
Acute allergic reaction_17
Acute allergic reaction_18
Acute allergic reaction_19
Acute allergic reaction_20
Acute allergic reaction_21
Acute allergic reaction_22
Acute bacterial sinusitis (disorder)_1
Acute bacterial sinusitis (disorder)_2
Acute bacterial sinusitis (disorder)_3
Acute bacterial sinusitis (disorder)_4
Acute bacterial sinusitis (disorder)_5
Acute bacterial sinusitis (disorder)_6
Acute bacterial sinusitis (disorder)_7
Acute bacterial sinusitis (disorder)_8
Acute bacterial sinusitis (disorder)_9
Acute bacterial sinusitis (disorder)

Bullet wound_13
Bullet wound_14
Bullet wound_15
Bullet wound_16
Bullet wound_17
Bullet wound_18
Bullet wound_19
Bullet wound_20
Bullet wound_21
Bullet wound_22
Cardiac Arrest_1
Cardiac Arrest_2
Cardiac Arrest_3
Cardiac Arrest_4
Cardiac Arrest_5
Cardiac Arrest_6
Cardiac Arrest_7
Cardiac Arrest_8
Cardiac Arrest_9
Cardiac Arrest_10
Cardiac Arrest_11
Cardiac Arrest_12
Cardiac Arrest_13
Cardiac Arrest_14
Cardiac Arrest_15
Cardiac Arrest_16
Cardiac Arrest_17
Cardiac Arrest_18
Cardiac Arrest_19
Cardiac Arrest_20
Cardiac Arrest_21
Cardiac Arrest_22
Child attention deficit disorder_1
Child attention deficit disorder_2
Child attention deficit disorder_3
Child attention deficit disorder_4
Child attention deficit disorder_5
Child attention deficit disorder_6
Child attention deficit disorder_7
Child attention deficit disorder_8
Child attention deficit disorder_9
Child attention deficit disorder_10
Child attention deficit disorder_11
Child attention deficit disorder_12
Child attention deficit disord

Concussion with loss of consciousness_16
Concussion with loss of consciousness_17
Concussion with loss of consciousness_18
Concussion with loss of consciousness_19
Concussion with loss of consciousness_20
Concussion with loss of consciousness_21
Concussion with loss of consciousness_22
Concussion with no loss of consciousness_1
Concussion with no loss of consciousness_2
Concussion with no loss of consciousness_3
Concussion with no loss of consciousness_4
Concussion with no loss of consciousness_5
Concussion with no loss of consciousness_6
Concussion with no loss of consciousness_7
Concussion with no loss of consciousness_8
Concussion with no loss of consciousness_9
Concussion with no loss of consciousness_10
Concussion with no loss of consciousness_11
Concussion with no loss of consciousness_12
Concussion with no loss of consciousness_13
Concussion with no loss of consciousness_14
Concussion with no loss of consciousness_15
Concussion with no loss of consciousness_16
Concussion with no

Fetus with chromosomal abnormality_6
Fetus with chromosomal abnormality_7
Fetus with chromosomal abnormality_8
Fetus with chromosomal abnormality_9
Fetus with chromosomal abnormality_10
Fetus with chromosomal abnormality_11
Fetus with chromosomal abnormality_12
Fetus with chromosomal abnormality_13
Fetus with chromosomal abnormality_14
Fetus with chromosomal abnormality_15
Fetus with chromosomal abnormality_16
Fetus with chromosomal abnormality_17
Fetus with chromosomal abnormality_18
Fetus with chromosomal abnormality_19
Fetus with chromosomal abnormality_20
Fetus with chromosomal abnormality_21
Fetus with chromosomal abnormality_22
First degree burn_1
First degree burn_2
First degree burn_3
First degree burn_4
First degree burn_5
First degree burn_6
First degree burn_7
First degree burn_8
First degree burn_9
First degree burn_10
First degree burn_11
First degree burn_12
First degree burn_13
First degree burn_14
First degree burn_15
First degree burn_16
First degree burn_17
First degr

History of disarticulation at wrist (situation)_21
History of disarticulation at wrist (situation)_22
History of lower limb amputation (situation)_1
History of lower limb amputation (situation)_2
History of lower limb amputation (situation)_3
History of lower limb amputation (situation)_4
History of lower limb amputation (situation)_5
History of lower limb amputation (situation)_6
History of lower limb amputation (situation)_7
History of lower limb amputation (situation)_8
History of lower limb amputation (situation)_9
History of lower limb amputation (situation)_10
History of lower limb amputation (situation)_11
History of lower limb amputation (situation)_12
History of lower limb amputation (situation)_13
History of lower limb amputation (situation)_14
History of lower limb amputation (situation)_15
History of lower limb amputation (situation)_16
History of lower limb amputation (situation)_17
History of lower limb amputation (situation)_18
History of lower limb amputation (situation

...

Non-small cell carcinoma of lung  TNM stage 4 (disorder)_2
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_3
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_4
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_5
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_6
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_7
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_8
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_9
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_10
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_11
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_12
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_13
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_14
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_15
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_16
Non-small cell carcinoma of lung  TNM stage 4 (disorder)_17
Non-small cell carcinoma of lung  TNM stage 4 (d

Pathological fracture due to osteoporosis (disorder)_20
Pathological fracture due to osteoporosis (disorder)_21
Pathological fracture due to osteoporosis (disorder)_22
Perennial allergic rhinitis_1
Perennial allergic rhinitis_2
Perennial allergic rhinitis_3
Perennial allergic rhinitis_4
Perennial allergic rhinitis_5
Perennial allergic rhinitis_6
Perennial allergic rhinitis_7
Perennial allergic rhinitis_8
Perennial allergic rhinitis_9
Perennial allergic rhinitis_10
Perennial allergic rhinitis_11
Perennial allergic rhinitis_12
Perennial allergic rhinitis_13
Perennial allergic rhinitis_14
Perennial allergic rhinitis_15
Perennial allergic rhinitis_16
Perennial allergic rhinitis_17
Perennial allergic rhinitis_18
Perennial allergic rhinitis_19
Perennial allergic rhinitis_20
Perennial allergic rhinitis_21
Perennial allergic rhinitis_22
Perennial allergic rhinitis with seasonal variation_1
Perennial allergic rhinitis with seasonal variation_2
Perennial allergic rhinitis with seasonal variation

Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_7
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_8
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_9
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_10
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_11
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_12
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_13
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_14
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_15
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_16
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_17
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_18
Primary small cell malignant neoplasm of lung  TNM stage 3 (disorder)_19
Primary small cell malignant neoplasm of lung  TNM sta

Rheumatoid arthritis_16
Rheumatoid arthritis_17
Rheumatoid arthritis_18
Rheumatoid arthritis_19
Rheumatoid arthritis_20
Rheumatoid arthritis_21
Rheumatoid arthritis_22
Rupture of appendix_1
Rupture of appendix_2
Rupture of appendix_3
Rupture of appendix_4
Rupture of appendix_5
Rupture of appendix_6
Rupture of appendix_7
Rupture of appendix_8
Rupture of appendix_9
Rupture of appendix_10
Rupture of appendix_11
Rupture of appendix_12
Rupture of appendix_13
Rupture of appendix_14
Rupture of appendix_15
Rupture of appendix_16
Rupture of appendix_17
Rupture of appendix_18
Rupture of appendix_19
Rupture of appendix_20
Rupture of appendix_21
Rupture of appendix_22
Rupture of patellar tendon_1
Rupture of patellar tendon_2
Rupture of patellar tendon_3
Rupture of patellar tendon_4
Rupture of patellar tendon_5
Rupture of patellar tendon_6
Rupture of patellar tendon_7
Rupture of patellar tendon_8
Rupture of patellar tendon_9
Rupture of patellar tendon_10
Rupture of patellar tendon_11
Rupture of pat

Tear of meniscus of knee_5
Tear of meniscus of knee_6
Tear of meniscus of knee_7
Tear of meniscus of knee_8
Tear of meniscus of knee_9
Tear of meniscus of knee_10
Tear of meniscus of knee_11
Tear of meniscus of knee_12
Tear of meniscus of knee_13
Tear of meniscus of knee_14
Tear of meniscus of knee_15
Tear of meniscus of knee_16
Tear of meniscus of knee_17
Tear of meniscus of knee_18
Tear of meniscus of knee_19
Tear of meniscus of knee_20
Tear of meniscus of knee_21
Tear of meniscus of knee_22
Third degree burn_1
Third degree burn_2
Third degree burn_3
Third degree burn_4
Third degree burn_5
Third degree burn_6
Third degree burn_7
Third degree burn_8
Third degree burn_9
Third degree burn_10
Third degree burn_11
Third degree burn_12
Third degree burn_13
Third degree burn_14
Third degree burn_15
Third degree burn_16
Third degree burn_17
Third degree burn_18
Third degree burn_19
Third degree burn_20
Third degree burn_21
Third degree burn_22
Tubal pregnancy_1
Tubal pregnancy_2
Tubal pregna

---
---

## Immunizations

In [110]:
batch = process_n_merge_processed (batch, 'immunizations_encounters.pkl')
print('Dimensions so far:', batch.shape)

Dimensions: (1287214, 544)
DTaP_1
DTaP_2
DTaP_3
DTaP_4
DTaP_5
DTaP_6
DTaP_7
DTaP_8
DTaP_9
DTaP_10
DTaP_11
DTaP_12
DTaP_13
DTaP_14
DTaP_15
DTaP_16
DTaP_17
DTaP_18
DTaP_19
DTaP_20
DTaP_21
DTaP_22
DTaP_23
DTaP_24
DTaP_25
DTaP_26
DTaP_27
DTaP_28
DTaP_29
DTaP_30
DTaP_31
DTaP_32
DTaP_33
DTaP_34
HPV  quadrivalent_1
HPV  quadrivalent_2
HPV  quadrivalent_3
HPV  quadrivalent_4
HPV  quadrivalent_5
HPV  quadrivalent_6
HPV  quadrivalent_7
HPV  quadrivalent_8
HPV  quadrivalent_9
HPV  quadrivalent_10
HPV  quadrivalent_11
HPV  quadrivalent_12
HPV  quadrivalent_13
HPV  quadrivalent_14
HPV  quadrivalent_15
HPV  quadrivalent_16
HPV  quadrivalent_17
HPV  quadrivalent_18
HPV  quadrivalent_19
HPV  quadrivalent_20
HPV  quadrivalent_21
HPV  quadrivalent_22
HPV  quadrivalent_23
HPV  quadrivalent_24
HPV  quadrivalent_25
HPV  quadrivalent_26
HPV  quadrivalent_27
HPV  quadrivalent_28
HPV  quadrivalent_29
HPV  quadrivalent_30
HPV  quadrivalent_31
HPV  quadrivalent_32
HPV  quadrivalent_33
HPV  quadrivalent_34
Hep A

pneumococcal polysaccharide vaccine  23 valent_29
pneumococcal polysaccharide vaccine  23 valent_30
pneumococcal polysaccharide vaccine  23 valent_31
pneumococcal polysaccharide vaccine  23 valent_32
pneumococcal polysaccharide vaccine  23 valent_33
pneumococcal polysaccharide vaccine  23 valent_34
rotavirus  monovalent_1
rotavirus  monovalent_2
rotavirus  monovalent_3
rotavirus  monovalent_4
rotavirus  monovalent_5
rotavirus  monovalent_6
rotavirus  monovalent_7
rotavirus  monovalent_8
rotavirus  monovalent_9
rotavirus  monovalent_10
rotavirus  monovalent_11
rotavirus  monovalent_12
rotavirus  monovalent_13
rotavirus  monovalent_14
rotavirus  monovalent_15
rotavirus  monovalent_16
rotavirus  monovalent_17
rotavirus  monovalent_18
rotavirus  monovalent_19
rotavirus  monovalent_20
rotavirus  monovalent_21
rotavirus  monovalent_22
rotavirus  monovalent_23
rotavirus  monovalent_24
rotavirus  monovalent_25
rotavirus  monovalent_26
rotavirus  monovalent_27
rotavirus  monovalent_28
rotavirus

---
---

## Save

Delete the batch table so far to liberate memory for processing the next table - `observations`.

In [111]:
batch.to_pickle('batch1.pkl')
del batch

## Observations

In [112]:
observations = load_table('observations_encounters.pkl')

Dimensions: (62209964, 64)


In [120]:
display(observations.head())

,Abuse Status [OMAHA],Alzheimer's disease (disorder),American house dust mite IgE Ab in Serum,Are you covered by health insurance or some other kind of health care plan [PhenX],Body Height,Body Mass Index,Body Weight,Calcium,Carbon Dioxide,Cardiac Arrest,...,Stroke,Systolic Blood Pressure,Total Cholesterol,Total score [MMSE],Triglycerides,Urea Nitrogen,Walnut IgE Ab in Serum,Wheat IgE Ab in Serum,White oak IgE Ab in Serum,PATIENT
0,0,0,0,0,2477883,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe
1,0,0,0,0,1822198,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36d131ee-dd5b-4acb-acbe-19961c32c099
2,0,0,0,0,0,0,2477883,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe
3,0,0,0,0,0,0,1822198,0,0,0,...,0,0,0,0,0,0,0,0,0,36d131ee-dd5b-4acb-acbe-19961c32c099
4,0,0,0,0,0,2477883,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33f33990-ae8b-4be8-938f-e47ad473abfe


In [ ]:
batch = pd.read_pickle('batch1.pkl')

In [118]:
cols2exclude = list(batch.columns)

In [122]:
df2 = patient_subset_index_explode(batch, observations)

subsetting...
Subset table dimensions: (4656661, 64)


C:\Users\Admin\AppData\Local\Temp\ipykernel_10776\87141843.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab['encounter_n'] = tab.groupby('PATIENT').cumcount()+1


In [125]:
del observations

In [127]:
print('Current consolidated table dimensions:', batch.shape)

Current consolidated table dimensions: (119293, 6472)


In [128]:
batch = batch_merge (batch, df2)

Merging...
Merged!
New dimensions: (119293, 21655)


In [129]:
del df2

In [ ]:
batch = tidy_up (batch, cols2exclude)

Filling up with 0s
